# autodl example

## 1. 创建客户端

In [1]:
import pandas as pd
import autodl

In [2]:
token = open('../.autodl_token', 'r').read()
client = autodl.AutoDLElasticDeployment(token)

## 2. 获取私有镜像列表

In [3]:
images = client.get_images(page_index=1, page_size=10)
pd.DataFrame(images)

,id,image_name,image_uuid
0,549274,hq_det,image-8663c9b91d
1,541913,vla-0521-idp3,image-4af49d99fe


## 3. 获取当前部署列表

In [4]:
deployments = client.get_deployments()
pd.DataFrame(deployments)

,id,uid,uuid,name,deployment_type,status,region_sign,dc_list,gpu_name_set,created_at
0,2459239,673024,cdb3170133,API测试部署,ReplicaSet,running,,[westDC2],[RTX 4090],2025-07-03T21:43:26+08:00
1,2459018,673024,c776a3a3a0,API测试部署,ReplicaSet,running,,[westDC2],[RTX 4090],2025-07-03T20:37:47+08:00


## 4. 获取GPU库存

In [5]:
# 获取西北企业区GPU库存 (CUDA 11.8)
region_sign = autodl.AutoDLConstants.REGIONS["西北企业区(推荐)"]
stock = client.get_gpu_stock(region_sign, 118)
pd.DataFrame(stock)

,gpu_type,total_gpu_num,idle_gpu_num,chip_corp,cpu_arch
0,vGPU-32GB,306,131,nvidia,x86
1,vGPU-48GB,56,6,nvidia,x86
2,RTX 4090D,300,35,nvidia,x86
3,RTX 4090,483,69,nvidia,x86
4,RTX 3090,168,5,nvidia,x86
5,RTX 3080x2,40,6,nvidia,x86


## 5. 创建部署

In [6]:
deployment_uuid =client.create_replicaset_deployment(
    name='API测试部署',
    image_uuid=autodl.AutoDLConstants.BASE_IMAGES['PyTorch 2.0.0 (CUDA 11.8)'],
    gpu_name_set=['RTX 4090'],
    replica_num=1,
    gpu_num=1,
    cpu_num_from=1,
    cpu_num_to=100,
    memory_size_from=1,
    memory_size_to=256,
    dc_list=['westDC2'],
    cuda_v_from=118,
    cmd='jupyter lab --ip=0.0.0.0 --port=8888 --no-browser --allow-root',
    env_vars={'JUPYTER_TOKEN': 'test123'},
    price_from=10,
    price_to=9000,
    reuse_container=True
)
deployment_uuid

'cdb3170133'

## 6. 管理部署容器

### 获取部署列表

In [7]:
deployments = client.get_deployments()

In [8]:
pd.DataFrame(deployments)

,id,uid,uuid,name,deployment_type,status,region_sign,dc_list,gpu_name_set,created_at
0,2459239,673024,cdb3170133,API测试部署,ReplicaSet,running,,[westDC2],[RTX 4090],2025-07-03T21:43:26+08:00
1,2459018,673024,c776a3a3a0,API测试部署,ReplicaSet,running,,[westDC2],[RTX 4090],2025-07-03T20:37:47+08:00
2,2459008,673024,8d358fbd8c,API测试部署,ReplicaSet,running,,[westDC2],[RTX 4090],2025-07-03T20:35:16+08:00


In [9]:
deployment_uuids = [deployment['uuid'] for deployment in deployments]
deployment_uuids

['cdb3170133', 'c776a3a3a0', '8d358fbd8c']

### 查询容器事件

In [10]:
query_container_events = client.query_container_events(deployment_uuids[-1])
display(pd.DataFrame(query_container_events))

,deployment_container_uuid,status,created_at
0,5dec112cb6-314744b79a-f1156c1ed8-32bdb0791,running,2025-07-03T20:35:20+08:00
1,5dec112cb6-314744b79a-f1156c1ed8-32bdb0791,starting,2025-07-03T20:35:17+08:00


### 查询容器

In [11]:
containers = client.query_containers(deployment_uuids[-1])
pd.DataFrame(containers)

,list,page_index,page_size,offset,max_page,result_total,page
0,"{'id': 29174368, 'machine_id': '314744b79a', '...",1,100,0,1,1,1


### 停止某容器

In [12]:
client.stop_deployment(deployment_uuids[-1])

True

### 设置副本数量

In [13]:
client.set_replicas(deployment_uuids[-1], 2)

True

### 停止部署

In [14]:
client.stop_deployment(deployment_uuids[-1])

True

### 删除部署

In [15]:
client.delete_deployment(deployment_uuids[-1])

True

In [16]:
pd.DataFrame(client.get_deployments())

,id,uid,uuid,name,deployment_type,status,region_sign,dc_list,gpu_name_set,created_at
0,2459239,673024,cdb3170133,API测试部署,ReplicaSet,running,,[westDC2],[RTX 4090],2025-07-03T21:43:26+08:00
1,2459018,673024,c776a3a3a0,API测试部署,ReplicaSet,running,,[westDC2],[RTX 4090],2025-07-03T20:37:47+08:00


### 设置调度黑名单

In [17]:
# client.set_scheduling_blacklist("xxx", 10, "test")

### 获取生效中的调度黑名单

In [18]:
# client.get_scheduling_blacklist(deployment_uuids[-1])

### 获取弹性部署GPU库存

In [19]:
stock = client.get_gpu_stock(region_sign, 117, 128)
pd.DataFrame(stock)

,gpu_type,total_gpu_num,idle_gpu_num,chip_corp,cpu_arch
0,vGPU-32GB,306,131,nvidia,x86
1,vGPU-48GB,56,6,nvidia,x86
2,RTX 4090D,300,35,nvidia,x86
3,RTX 4090,483,68,nvidia,x86
4,RTX 3090,168,5,nvidia,x86
5,RTX 3080x2,40,6,nvidia,x86


### 获取已购时长包数据

In [ ]:
get_ddp_overview = client.get_ddp_overview(deployment_uuids[-1])
pd.DataFrame(get_ddp_overview)